python 3.13.1

Get Naive model on the same tickers used for my sample

In [1]:
import requests
import yfinance as yf
import pandas as pd
import random
import numpy as np
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_squared_error

c:\Users\Shane\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
smapes_df_path = r"C:\Users\Shane\Desktop\2024.06.27_-_Data_Science\2024.10.13 - Portfolio Projects\2024.12.04 - Swing Ticker\SMAPE Model Eval\all_models_smape.csv"

In [3]:
smapes_df = pd.read_csv(smapes_df_path)
smapes_df.head()

,ticker,volatility,avg_price,len,smape_standard,smape_tuned,winsorized_pct
0,WPP,Medium,32.136355,9355,0.270578,0.075833,0.364725
1,CVGW,Medium,31.071787,5681,0.162737,0.163886,0.363316
2,DFH,Medium,22.145518,1023,0.305700,0.239105,0.418377
3,RCEL,Medium-High,13.289237,3209,0.915579,0.627327,0.267061
4,C,Medium-Low,79.326248,12132,0.325752,0.241818,0.368035


In [4]:
tickers = smapes_df['ticker'].tolist()
tickers[:5]

['WPP', 'CVGW', 'DFH', 'RCEL', 'C']

In [5]:
def load_data(ticker):
    """
    Downloads historical market data for a given ticker symbol.

    Parameters:
    ticker (str): The ticker symbol of the stock to download data for.

    Returns:
    pd.DataFrame: A DataFrame containing the historical market data for the specified ticker.
    """
    data = yf.download(ticker, period='max') # returns relevant data in df
    data.reset_index(inplace=True) # reset multindex, output is index list of tuples
    cols = list(data.columns) # convert index to list
    cols[0] = ('Date', '') 
    cols = [i[0] for i in cols] # return first element of cols tuples
    data.columns = cols # set as column names
    data['Date'] = pd.to_datetime(data['Date']).dt.date
    data.Date = data.Date.astype('datetime64[ns]')
    return data

In [ ]:
def get_volatility(data):
    """
    Calculate the volatility of daily returns and categorize it into levels.

    This function computes the annualized volatility of daily returns from the input data.
    It then categorizes the volatility into one of five levels: "Low", "Medium-Low", "Medium", 
    "Medium-High", or "High" based on predefined thresholds.

    Parameters:
    data (DataFrame): A pandas DataFrame containing a 'daily_returns' column with daily return values.

    Returns:
    tuple: A tuple containing the volatility category (str) and the calculated volatility (float).
    """
    volatility = data.daily_returns.std() * np.sqrt(252)
    if volatility < 0.2:
        category = "Low"
        percentiles=(0.15, 0.85)
    elif volatility < 0.4:
        category = "Medium-Low"
        percentiles=(0.1, 0.9)
    elif volatility < 0.6:
        category = "Medium"
        percentiles=(0.1, 0.9)
    elif volatility < 0.8:
        category = "Medium-High"
        percentiles=(0.05, 0.95)
    else:
        category = "High"
        percentiles=(0.05, 0.95)
    return category, volatility, percentiles

In [ ]:
def get_period_params(data, volatility):
    """
    Determine the training, period unit, and forecast periods based on data length and volatility.

    This function calculates the appropriate training period, period unit, and forecast period 
    based on the length of the input data and the given volatility. If the data length is less 
    than 8 years, the period unit and forecast period are set to a quarter of the data length. 
    Otherwise, the period unit is set to 1 year, and the forecast period is set to 1 year. 
    The training period is adjusted based on the volatility.

    Parameters:
    data (DataFrame): A pandas DataFrame containing the data.
    volatility (float): The calculated annualized volatility.

    Returns:
    tuple: A tuple containing:
        - train_period (int): The length of the training period.
        - period_unit (int): The length of the period unit.
        - forecast_period (int): The length of the forecast period.
    """
    if len(data)/365 < 8:
        period_unit = int(len(data)/4)
        forecast_period = period_unit
        train_period = len(data)
    else:
        period_unit = 365
        forecast_period = period_unit
        train_period = forecast_period * 4 if volatility < 0.6 else forecast_period * 8
    return train_period, period_unit, forecast_period

In [ ]:
# Lambda function for performing cross-validation with specified parameters


cv_func = lambda model_name, train_period, period_unit, forecast_period: cross_validation(model_name, 
                                              initial=f'{train_period} days', 
                                              period=f'{period_unit} days', 
                                              horizon=f'{forecast_period} days', 
                                              parallel="processes")

In [10]:
def naive_forecast(history):
    """
    Performs a naive forecast using the last value in the history.

    Parameters:
    history (pd.Series): Time series data to use as history.

    Returns:
    float: The naive forecast (last value of history).
    """
    return history.iloc[-1]

In [11]:
def naive_cross_validation(data, initial, period, horizon):
    """
    Performs cross-validation for a naive forecasting model.

    Parameters:
    data (pd.DataFrame): Time series data with 'Date' and 'Close' columns.
    initial (int): Initial training period length in days.
    period (int): Period between cutoffs in days.
    horizon (int): Forecast horizon in days.

    Returns:
    pd.DataFrame: DataFrame containing actual values, forecasts, and cutoffs.
    """
    initial_date = data['Date'].iloc[0]
    cutoffs = pd.to_datetime([initial_date + pd.Timedelta(days=initial + i * period)
                               for i in range(1 + (len(data) - initial - horizon) // period)])

    periods_list = []
    for cutoff in cutoffs:
        train_df = data[data['Date'] <= cutoff].copy()
        test_df = data[(data['Date'] > cutoff) & (data['Date'] <= cutoff + pd.Timedelta(days=horizon))].copy()

        # if len(test_df) == 0: # Skip if no data in the horizon
        #     continue

        history = train_df['Close']
        forecasts = [naive_forecast(history)] * len(test_df) # Naive forecast for all points in horizon

        test_df['forecast'] = forecasts
        test_df['cutoff'] = cutoff.date() # Store cutoff date

        periods_list.append(test_df)

    df_cv = pd.concat(periods_list).reset_index(drop=True)
    return df_cv

In [12]:
def calculate_smape(cv_results):
    """
    Calculates smape from cross-validation results DataFrame.

    Parameters:
    cv_results (pd.DataFrame): DataFrame from naive_cross_validation.

    Returns:
    float: smape value.
    """
    if cv_results.empty:
        return np.nan  # Return NaN if no cross-validation results

    return np.sqrt(mean_squared_error(cv_results['Close'], cv_results['forecast']))

In [13]:
def get_naive_smape_for_tickers(tickers):
    """
    Calculates naive model smape for a list of stock tickers.

    Parameters:
    tickers (list): List of stock ticker symbols.

    Returns:
    pd.DataFrame: DataFrame with ticker symbols and their naive model smapes.
    """
    naive_smape_results = []
    for ticker in tickers:
        data = load_data(ticker)
        data['daily_returns'] = data.Close.pct_change()
        category, volatility, percentiles = get_volatility(data)
        train_period, period_unit, forecast_period = get_period_params(data, volatility)

        initial_days = train_period
        period_days = period_unit
        horizon_days = forecast_period

        cv_results_naive = naive_cross_validation(
            data[['Date', 'Close']], # Ensure only Date and Close are passed
            initial=period_days,
            period=period_days,
            horizon=horizon_days
        )
        smape_naive = calculate_smape(cv_results_naive)
        naive_smape_results.append({'ticker': ticker, 'naive_smape': smape_naive})
        print(f"Naive smape for {ticker}: {smape_naive:.4f}")

    return pd.DataFrame(naive_smape_results)

In [14]:
naive_smape_df = get_naive_smape_for_tickers(tickers)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Naive smape for WPP: 4.8835
Naive smape for CVGW: 4.1601


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Naive smape for DFH: 5.9980
Naive smape for RCEL: 5.6555


[*********************100%***********************]  1 of 1 completed


...

Naive smape for PZA: 1.0403
Naive smape for RPTX: 4.8855


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Naive smape for PBF: 8.8848
Naive smape for DRN: 2.6732


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Naive smape for ULCC: 3.0466
Naive smape for AZPN: 9.3325


In [15]:
naive_smape_df.head()

,ticker,naive_smape
0,WPP,4.883492
1,CVGW,4.160057
2,DFH,5.998029
3,RCEL,5.655529
4,C,29.475851


In [16]:
naive_smape_df.naive_smape.mean()

np.float64(25262.187914633916)

In [ ]:
# merge with smapes_df

smapes_df = pd.merge(left=smapes_df, 
         right=naive_smape_df,
         on='ticker')

In [ ]:
# export

smapes_df.to_csv(smapes_df_path)